# Lab | Data Aggregation and Filtering

Objective: Use Python programming techniques to filter and order data subsets, group data by one or more variables, and use aggregation functions to obtain insights, and derive meaningful conclusions from the data.

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing first data cleaning, formatting, and structuring.

Start by reading the dataset, standardizing column names, dealing with null values and creating the month column. You can do so by using the functions you developed in previous labs, or by reading the clean dataset you saved in the last lab `data structuring and combining`.

Then, answer the following questions:

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the median customer lifetime value by education level and gender.

5. Find the maximum, minimum, and average number of open complaints by state and policy type.

6. The marketing team wants to analyze the number of policies sold by state and month.

Present the data in a table where the months are arranged as columns and the states are arranged as rows.

7. Display a new DataFrame that contains the number of policies sold by month by state for the top 3 states with the highest number of policies sold.

*Hint:* 
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

8. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [138]:
# your code goes here
import pandas as pd

df= pd.read_csv("df_final.csv")
df.head(1)

,unnamed 0,customer,state,customer lifetime value,response,coverage,education,effective to date,employmentstatus,gender,...,number of policies,policy type,policy,renew offer type,sales channel,total claim amount,vehicle class,vehicle size,vehicle type,month
0,0,DK49336,Arizona,4809.21696,No,Basic,College,2011-02-18,Employed,M,...,9,Corporate Auto,Corporate L3,Offer3,Agent,292.8,Four-Door Car,Medsize,NaN,February


1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [146]:
df_yes = df[(df["total claim amount"]>=1000) & (df["response"]=="Yes")]#Boolean mask to filter 1
df_yes.head(1)

,unnamed 0,customer,state,customer lifetime value,response,coverage,education,effective to date,employmentstatus,gender,...,number of policies,policy type,policy,renew offer type,sales channel,total claim amount,vehicle class,vehicle size,vehicle type,month
189,189,OK31456,California,11009.13049,Yes,Premium,Bachelor,2011-01-24,Employed,F,...,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.4,Luxury Car,Medsize,NaN,January


2. Analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign.

In [130]:
df_yes_1 = df[df["response"]=="Yes"]
df_yes_1 = df_yes_1.groupby(["gender","policy type"]).agg({"total claim amount":"mean"}).round()
df_yes_1

total claim amount
gender policy type                       
F      Corporate Auto               432.0
       Personal Auto                453.0
       Special Auto                 460.0
M      Corporate Auto               410.0
       Personal Auto                456.0
       Special Auto                 415.0

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [136]:
customers_per_state = df["state"].value_counts()
customers_per_state[customers_per_state>500]



state
California    3035
Oregon        2517
Arizona       1655
Nevada         863
Washington     774
Name: count, dtype: int64

4. Find the median customer lifetime value by education level and gender.

In [142]:
avg_lv = df.groupby(["gender", "education"]).agg({"customer lifetime value":"median"}).round()
avg_lv

customer lifetime value
gender education                                    
F      Bachelor                               5678.0
       College                                5622.0
       Doctor                                 5332.0
       High School or Below                   6044.0
       Master                                 5801.0
M      Bachelor                               5556.0
       College                                5990.0
       Doctor                                 5621.0
       High School or Below                   6177.0
       Master                                 5618.0

5. Find the maximum, minimum, and average number of open complaints by state and policy type.

In [149]:
df_compl = df.groupby(["state","policy type"],as_index=False).agg({"number of open complaints":["max","min","mean"]})
df_compl

state     policy type number of open complaints               
                                                     max  min      mean
0      Arizona  Corporate Auto                       5.0  0.0  0.396104
1      Arizona   Personal Auto                       5.0  0.0  0.392974
2      Arizona    Special Auto                       3.0  0.0  0.260274
3   California  Corporate Auto                       4.0  0.0  0.312955
4   California   Personal Auto                       5.0  0.0  0.397759
5   California    Special Auto                       5.0  0.0  0.435185
6       Nevada  Corporate Auto                       5.0  0.0  0.459459
7       Nevada   Personal Auto                       5.0  0.0  0.411955
8       Nevada    Special Auto                       5.0  0.0  0.545455
9       Oregon  Corporate Auto                       4.0  0.0  0.345098
10      Oregon   Personal Auto                       5.0  0.0  0.371823
11      Oregon    Special Auto                       5.0  0.0  0.314050
12  Washington  Corporate Auto                       5.0  0.0  0.523179
13  Washington   Personal Auto                       5.0  0.0  0.390845
14  Washington    Special Auto                       4.0  0.0  0.551724

6. The marketing team wants to analyze the number of policies sold by state and month.

Present the data in a table where the months are arranged as columns and the states are arranged as rows.

In [242]:
df.pivot_table(index="state",
               columns="month",
               values="number of policies",
               aggfunc="sum")

month,February,January
state,,
Arizona,2384,2658
California,4218,4791
Nevada,1072,1330
Oregon,3469,4018
Washington,1074,1184


7. Display a new DataFrame that contains the number of policies sold by month by state for the top 3 states with the highest number of policies sold.

*Hint:* 
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

In [233]:
final_boss = df.groupby(["state","month"],as_index=False).agg({"number of policies":"sum"}).sort_values("number of policies",ascending=False)
final_boss

,state,month,number of policies
3,California,January,4791
2,California,February,4218
7,Oregon,January,4018
6,Oregon,February,3469
1,Arizona,January,2658
0,Arizona,February,2384
5,Nevada,January,1330
9,Washington,January,1184
8,Washington,February,1074
4,Nevada,February,1072


In [240]:
top_states = df.groupby(["state"],as_index=False).agg({"number of policies":"sum"}).nlargest(3,columns="number of policies")["state"]
top_states

1    California
3        Oregon
0       Arizona
Name: state, dtype: object

In [241]:
final_boss[final_boss["state"].isin(top_states)]

,state,month,number of policies
3,California,January,4791
2,California,February,4218
7,Oregon,January,4018
6,Oregon,February,3469
1,Arizona,January,2658
0,Arizona,February,2384


8. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

In [243]:
df.head(1)

,unnamed 0,customer,state,customer lifetime value,response,coverage,education,effective to date,employmentstatus,gender,...,number of policies,policy type,policy,renew offer type,sales channel,total claim amount,vehicle class,vehicle size,vehicle type,month
0,0,DK49336,Arizona,4809.21696,No,Basic,College,2011-02-18,Employed,M,...,9,Corporate Auto,Corporate L3,Offer3,Agent,292.8,Four-Door Car,Medsize,NaN,February


In [258]:
melt= df.groupby(["sales channel","response"]).agg({"customer":"count"})
melt
#melted = pd.melt(melt, id_vars=["reponse"])

customer
sales channel response          
Agent         No            2730
              Yes            639
Branch        No            2209
              Yes            282
Call Center   No            1521
              Yes            186
Web           No            1126
              Yes            151